In [ ]:
#all importing Stuff

import pandas as pd
import numpy as np
from xgboost import *
from lightgbm import *
from sklearn.ensemble import *

In [ ]:
UseOriginalDataset=False
copies=1
seed=0

In [ ]:
def get_combined_data(df,original,copies):
    for _ in range(copies):
        df = pd.concat([df,original],ignore_index=True)
    return df

In [ ]:
train=pd.read_csv("/kaggle/input/playground-series-s4e6/train.csv").drop("id",axis=1)
test=pd.read_csv("/kaggle/input/playground-series-s4e6/test.csv").drop("id",axis=1)
submission=pd.read_csv("/kaggle/input/playground-series-s4e6/sample_submission.csv")
origin=pd.read_csv("//kaggle/input/predict-students-dropout-and-academic-success/data.csv")

#Target
target="Target"

print("Train Dataset =",train.shape)
display(train)
print("Test Dataset =",test.shape)
display(test)
print("Original Dataset =",origin.shape)
display(origin)

In [ ]:
if UseOriginalDataset:
    df=get_combined_data(train,origin,copies)
else:
    df=train.copy()


In [ ]:
cat_feats=['Marital status', 'Application mode', 'Course','Daytime/evening attendance', 'Previous qualification', 
           'Nationality',"Mother's qualification", "Father's qualification",'Displaced', 'Educational special needs', 'Debtor',
       'Tuition fees up to date', 'Gender', 'Scholarship holder',
       'Age at enrollment', 'International']

In [ ]:
df[target] = df[target].map({'Graduate':2, 'Dropout': 0,'Enrolled':1})
df[target]=df[target].astype(int)
train[target] = train[target].map({'Graduate':2, 'Dropout': 0,'Enrolled':1})
train[target]=train[target].astype(int)
origin[target] = origin[target].map({'Graduate':2, 'Dropout': 0,'Enrolled':1})
origin[target]=origin[target].astype(int)

In [ ]:
lgb_params={'objective': 'multiclass',
                   'n_estimators': 770,
                   'learning_rate': 0.023635608862537988,
                   'data_sample_strategy': 'goss',
                   'feature_fraction': 0.2841670798274386,
                   'tree_learner': 'feature',
                   'lambda_l1': 1.9832702497881635e-07, 'lambda_l2': 0.41501187037770015,
                   'num_leaves': 888, 'max_depth': 9, 'colsample_bytree': 0.7804414817467027, 'min_child_samples': 126,
                   'min_sum_hessian_in_leaf': 5.833100626676496, 'min_gain_to_split': 0.8004030774733167, 'max_bin': 169,
                   'top_rate': 0.6338213165561601}

xgb_params={'max_depth': 5,
             'lambda': 0.04511179009306839,
             'alpha': 0.2356856077657757,
             'colsample_bytree': 0.9684456706095538,
             'subsample': 0.7387404229360499,
             'learning_rate': 0.01952985763275396,
             'n_estimators': 2665,
             'grow_policy': 'depthwise',
             'sampling_method': 'gradient_based',
             'gamma': 0.0010790912320170717,
             'min_child_weight': 79,
             'max_leaves': 1457,
             'max_delta_step': 3.9096375543686004,
             'objective':'multi:softmax',
             'eval_metric':'mlogloss',
             'random_state':42,
             'device':'cuda',
             'num_class':3,
            }
xgb=XGBClassifier(**xgb_params)
lgb=LGBMClassifier(**lgb_params,verbose=-1,seed=42,num_class=3
                       ,n_jobs=4,
                    categorical_feature=[0, 1, 3, 4, 5, 7, 8, 9, 13, 14, 15, 16, 17, 18, 19, 20]
                      )

In [ ]:
estimators = [
    ('lgb',BaggingClassifier(estimator=lgb,n_estimators=100)),
   ('xgb',BaggingClassifier(estimator=xgb,n_estimators=100)),
]

final_model=VotingClassifier(estimators=estimators,voting='soft')

final_model.fit(df.drop(target,axis=1),df[target])

preds=final_model.predict(test)


In [ ]:

submission[target]=preds
submission[target]=submission[target].map({2:'Graduate', 0:'Dropout',1:'Enrolled'})
submission[target]=submission[target].astype(str)
submission
display(submission)
submission.to_csv("submission.csv",index=False)

### AutoML Solution, it was combined to this https://www.kaggle.com/code/gauravduttakiit/pss4e6-flaml-roc-auc-ovo notebook with mode ensembling...!